In [16]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from sklearn.linear_model import LinearRegression

## Temperatura

In [4]:
table1 = pd.read_csv('Temperatures.csv')

In [61]:
# datos
pais = 'ARG'
# df = table1['year', 'temperatura'].groupby('year').mean().reset_index()
# df = table1[table1['codigo'] == pais]
df = table1[['year', 'temperatura']].groupby('year').mean().reset_index()
avg = df[df['temperatura'] < 2000]['temperatura'].mean()
maxmin = df[(df['temperatura'] == df['temperatura'].max()) | (df['temperatura'] == df['temperatura'].min())]

# Regresión linear
x_min_model_fit = 1901
X = df[df['year'] > x_min_model_fit]['year'].values.reshape(-1,1)
y = df[df['year'] > x_min_model_fit]['temperatura'].values.reshape(-1,1)
model = LinearRegression()
model.fit(X,y)
X_range = np.arange(1901, 2031).reshape(-1,1)
X_predict = np.arange(x_min_model_fit, 2031).reshape(-1,1)

#---------Colores
comp = '#636EFA'
prom = '#00CC96'
regl = '#AB63FA'
extr = '#D62728'
limM = '#565656'

# Graficamos
fig = go.Figure()
fig.update_layout(title_text = 'Comportamiento del Promedio')
to_graph = ['temp', 'dif'][0]
if to_graph == 'temp':
    fig.add_trace(go.Scatter(x = X_range.reshape(-1), y = avg + 1.5*np.zeros_like(X_range.reshape(-1)) + 1.5, name = 'Límite {:.2f}ºC'.format(avg+1.5), line = {'color' : limM, 'dash':'dot'}))
    fig.add_trace(go.Scatter(x = df['year'], y = df['temperatura'], name = 'Comportamiento', line = {'color' : comp}))
    fig.add_trace(go.Scatter(x = maxmin['year'], y = maxmin['temperatura'], mode = 'markers', name = 'Extremos', line = {'color' : extr}))
    fig.add_trace(go.Scatter(x = X_range.reshape(-1), y = avg*np.ones_like(X_range.reshape(-1)), name = 'Promedio {:.2f}ºC'.format(avg), line = {'color' : prom, 'dash':'dash'}))
    fig.add_trace(go.Scatter(x = X_predict.reshape(-1), y = model.predict(X_predict).reshape(-1), name = 'Regresión linear', line = {'color' : regl, 'dash':'dashdot'}))
    fig.update_layout(yaxis_title = 'Temperatura')
else:
    fig.add_trace(go.Scatter(x = df['year'], y = df['temperatura'].values - avg*np.ones(len(df)), name = 'Comportamiento', line = {'color' : comp}))
    fig.add_trace(go.Scatter(x = X_range.reshape(-1), y = np.zeros(len(X_range.reshape(-1))), name = 'Promedio (0ºC)', line = {'color' : prom, 'dash':'dash'}))
    fig.add_trace(go.Scatter(x = X_range.reshape(-1), y = 1.5*np.ones_like(X_range.reshape(-1)), name = 'Límite (1.5ºC)', line = {'color' : limM, 'dash':'dot'}))
    fig.add_trace(go.Scatter(x = X_range.reshape(-1), y = model.predict(X_range).reshape(-1) - avg*np.ones(len(X_range.reshape(-1))), name = 'Regresión linear', line = {'color' : regl, 'dash':'dashdot'}))
    fig.update_layout(taxis_title = 'Diferencia al promedio')
fig.show()

## Poporción de población con acceso a energía

In [62]:
table2 = pd.read_csv('datos_ONU.csv')
table2.head()

,Pais,Anio,intensidad_energetica_medida_en_terminos_de_energia_primaria_y_PBI,proporcion_de_la_poblacion_con_acceso_a_elecricidad,proporcion_de_la_poblacion_con_dependencia_primaria_a_energias_limpias,proporcion_de_energias_renovables_del_total_consumido
0,Argentina,2000,3.69,95.78329,95,9.85
1,Argentina,2001,3.59,95.51106,96,11.51
2,Argentina,2002,3.99,96.22887,96,11.62
3,Argentina,2003,3.97,96.44263,96,10.82
4,Argentina,2004,4.04,96.65350,96,9.29


In [88]:
num_c = 3
columnas = ['intensidad_energetica_medida_en_terminos_de_energia_primaria_y_PBI', 'proporcion_de_la_poblacion_con_acceso_a_elecricidad', 'proporcion_de_la_poblacion_con_dependencia_primaria_a_energias_limpias', 'proporcion_de_energias_renovables_del_total_consumido']
columna = columnas[num_c]
if num_c == 0:
    limite_v = None
else:
    limite_v = 1000
df = table2[['Anio', columna]].groupby('Anio').mean().reset_index()
avg = df[columna].mean()
maxmin = df[(df[columna] == df[columna].min()) | (df[columna] == df[columna].max())]

# Colores
comp = '#636EFA'
prom = '#00CC96'
regl = '#AB63FA'
extr = '#D62728'
limM = '#565656'

# Modelo
X = df['Anio'].values.reshape(-1,1)
y = df[columna].values.reshape(-1,1)
model = LinearRegression()
model.fit(X,y)
X_range = np.arange(2000, 2025).reshape(-1,1)

fig = go.Figure()
fig.update_layout(title_text = ' '.join(columna.split('_')).capitalize())
if limite_v:
    fig.add_trace(go.Scatter(x = X_range.reshape(-1), y = 100*np.ones_like(X_range.reshape(-1)), name = 'Límite 100%', line = {'color' : limM, 'dash':'dot'}))
fig.add_trace(go.Scatter(x = df['Anio'], y = df[columna], name = pais, line={'color':comp}))
fig.add_trace(go.Scatter(x = maxmin['Anio'], y = maxmin[columna], mode = 'markers', name = 'Extremos', line={'color':extr}))
fig.add_trace(go.Scatter(x = X_range.reshape(-1), y = avg*np.ones_like(X_range.reshape(-1)), name = 'Promedio', line={'color':prom, 'dash':'dash'}))
fig.add_trace(go.Scatter(x = X_range.reshape(-1), y = model.predict(X_range).reshape(-1), name = 'Regresión linear', line = {'color' : regl, 'dash':'dashdot'}))

fig.show()

In [41]:
order2019 = table2[(table2['Anio'] == 2019)][['Pais', columna]].sort_values(columna, ascending=False)
order2019

,Pais,proporcion_de_la_poblacion_con_acceso_a_elecricidad
19,Argentina,100.00000
59,Canadá,100.00000
79,Chile,100.00000
539,San Cristóbal y Nieves,100.00000
499,Dominica,100.00000
459,Barbados,100.00000
439,Bahamas,100.00000
419,Antigua y Barbuda,100.00000
379,Trinidad y Tobago,100.00000
399,Uruguay,99.90000


In [48]:
compl = len(order2019[order2019[columna] <= 95])
not_c = len(order2019[order2019[columna] > 95])
fig = px.pie(names = ['Cumplido', 'No cumplido'], values=[compl, not_c])
fig.show()

## Gráfico de proporción de energía renovable en el consumo final de energía

In [61]:
pais = 'México'
columna = 'proporcion_de_energias_renovables_del_total_consumido'
df = table2[table2['Pais'] == pais][['Anio', columna]]
avg = df[columna].mean()
maxmin = df[(df[columna] == df[columna].min()) | (df[columna] == df[columna].max())]

fig = go.Figure()
fig.add_trace(go.Scatter(x = df['Anio'], y = df[columna], name = pais))
fig.add_trace(go.Scatter(x = maxmin['Anio'], y = maxmin[columna], mode = 'markers', name = 'Extremos'))
fig.add_trace(go.Scatter(x = df['Anio'], y = avg*np.ones(len(df)), name = 'Promedio'))
fig.show()

In [50]:
order2019 = table2[(table2['Anio'] == 2019)][['Pais', columna]].sort_values(columna, ascending=False)
order2019

,Pais,proporcion_de_energias_renovables_del_total_consumido
219,Guatemala,62.92
399,Uruguay,60.76
339,Paraguay,60.09
299,Nicaragua,50.71
39,Brasil,47.57
239,Honduras,45.96
119,Costa Rica,35.76
99,Colombia,30.72
479,Belice,29.09
79,Chile,25.26


In [51]:
compl = len(order2019[order2019[columna] <= 95])
not_c = len(order2019[order2019[columna] > 95])
fig = px.pie(names = ['Cumplido', 'No cumplido'], values=[compl, not_c])
fig.show()